# The first **big** project
## In this project we will be using Pushover, but what is it?
- Pushover is a nifty tool for sending Push Notifications to your phone.
- It's super easy to set up and install.
- Simply visit - this [website](https://pushover.net/) and *Login and Signup* for a free account, and create an API key.
- Once you've signed up, on the home screen, click "Create an Application/API Token", and give it any name. Then, click *Create Application*.
- After that, add 2 lines to your `.env` file that you have.
  - `PUSHOVER_USER=`*put the key that's on the top right of your Pushover home screen, starts with a **u***
  - `PUSHOVER_TOKEN=`*put the key that shows up when you click into your new application called <whatever you named it>, the key starts with an **a***
- Finally, after you have saved your `.env` and loaded it in *python*, click "Add Phone, Table or Desktop" to install on your phone.


In [ ]:
from dotenv import load_dotenv
from openai import OpenAI
import json
import os
import requests
from pypdf import PdfReader
import gradio as gr

1. `dotenv` - needed for `load_dotenv( )`
2. `OpenAI` - for OpenAI API
3. `json` - better grouping/formatting of data 
4. `os` - accessing files on your computer
5. `requests` - requests API in which you provide an endpoint URL, from which API works, *more details below*.
6. `pypdf` - for reading pdf files
7. `gradio` - for providing GUI

### What is `requests`?
- It is a popular library in *python* used for making **HTTP requests**, or **web requests**.
- It is used in APIs, like `grok`, `deepseek`, etc.
- Here is how they are coded: -
  ``` python
  import requests
  from dotenv import load_dotenv
  import os
  load_dotenv()
  key = os.getenv("API_KEY")
  url = <url here>
  headers = {
    "Authorization": f"Bearer {key}"
    "Content-type": "application/json"
  }
  payload = {
    "model": <model here>
    "messages": [
        {"role": "user", "content": "Hi, how are you"}
    ]
    "stream": False
  }
  response = requests.post(url, headers=headers, json=payload)
  data = response.json()
  reply = data['choices'][0]['message']['content']
  print(reply)
  ```
- `url`, `headers`, `payload`: These variables set up the details of the request.
- `url` is the specific address of the API endpoint you're sending the request to.
- `headers` provides extra information with the request. The Authorization header uses your API key to authenticate the request, proving you have permission to access the API. The Content-type header tells the server that the data you're sending is in JSON format.
- `payload` is a Python dictionary that contains the core information for the request. It specifies the model you want to use, the messages that form the conversation (in this case, "Hi, how are you"), and whether the response should stream back one word at a time or all at once (False here means all at once).
- `response = requests.post(url, headers=headers, json=payload)`: This line sends the actual request to the API. requests.post is used because you are sending data (the payload) to the server.
- `data = response.json()`: This line takes the response from the server and converts its JSON content into a Python dictionary, making it easy to work with.
- `reply = data['choices'][0]['message']['content']`: This line navigates through the nested dictionary structure of the data to extract the AI's actual reply. It finds the first choice from the model and pulls out the text content of the message.
- `print(reply)`: Finally, this line prints the AI's response to your terminal.
- Now, let's continue with the usual things, you'll automatically recognize the `requests` library code.

In [ ]:
# The usual things

load_dotenv(override=True)
api = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api)

In [ ]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

- Now, what we have done is we have used our `.env` variables and given the `pushover_url` API link.
- And you guessed it, we will use it in `requests`.

In [ ]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
push("Hey there, this is a sample notification.")

- We have made a `push(message)` function, where we have used `requests` to post our `data` which is assigned to `payload`
- Then, we have called the function and the message will be printed on the device.

In [ ]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}
def record_unknown_question(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

- Now, what we have done here is made two functions, for recording 2 things from the user: -
    - `record_user_details(..)` : We are asking for interest and printing the user details which we will ask later.
    - `record_unknown_question(..)` : Anything else that the user asked and system couldn't answer.
- Both of these functions end with, `return {"recorded": "ok"}`, meaning that the functions will return that: "Ok, message has been recorded, let's continue and end this function"

In [ ]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provided it"
            }
            ,
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that's worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}

- The code above may seem complicated, but it's simple, let me tell you about it: -
  - We have defined a schema for a tool that records user details:
    - Must include email (a string).
    - May include name and notes (both are strings).
    - No other extra fields are allowed.
- We have used `json`, that we imported earlier, used it to define a *Python* Dictionary, with a **JSON** format.
- Now, you may be thinking, what is `json`, well great let me tell you.

#### `json` simplified
- It is a lightweight format to store and share data between systems.
- Here, we are sharing data between our *Python* program and the `Pushover` API.
- Rules:
  - Uses key-value pairs ("key": value).
  - Key must be in **double-quotes only**.
  - Values can be:
    - String (in `""`)
    - Number/Integer (`123`)
    - Boolean (`true`/`false`)
    - Null (`null`)
    - Array (`[1, 2, 3]`, in `[]`)
    - Object (`{"example": "data"}`)
  - No comments allowed.
  - Trailing commas (extra ones) aren't allowed
##### JSON = JavaScript Object Notation


In [ ]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record any question that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

- Here, we have given defined a schema that records unknown questions from the user:
  - `question` can only be a string.
  - If question is called, it is `required`, with `additionalProperties` being false, meaning, only one property, which is the `"question"`, is the only one and more can't be added.

In [ ]:
tools = [
    {"type": "function", "function": record_user_details_json},
    {"type": "function", "function": record_unknown_question_json}
]
# Now we have assigned those JSON properties to a variable, containing them as objects
# One object is for the normal user details while the other is for the unknown question

In [ ]:
tools

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        if tool_name == "record_user_details":
            result = record_user_details(**arguments)
        elif tool_name == "record_unknown_question":
            result = record_unknown_question(**arguments)
        results.append(
            {"role": "tool", "content": json.dumps(result), "tool_call_id": tool_call.id}
        )
    return results

- We have defined a function that handles tool calls and identifies them.
  - In this function we have added a `results` variable with no objects
  - Then we have made a `for` loop, that loads arguments from the `json` variables we made earlier.
  - Then we have made 2 big `if` statements:
    - `if tool_name == "record_user_details"`: if tool name is about recording user details, then run the corresponding function, with `json` arguments we assigned in a variable in this `for` loop.
    - `elif tool_name == "record_unknown_question"`: else, if tool name is about recording user's unknown question, then run the correspoinding function, again with the `json` arguments we assigned to a variable in this `for` loop.

In [ ]:
globals()["record_unknown_question"]("this is a really hard question")

- Then, the usual things, if not understanding then go learn it [here](2_lab__.ipynb).

In [ ]:
reader = PdfReader("samples/sample1.pdf")
chapter = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        chapter += text
with open("samples/sample1.pdf", "rb", encoding="utf-8") as f:
    summary = f.read()
name = "Structured Query Language (SQL)"

In [ ]:
system_prompt=f"""You are every student's favorite teacher who specializes in teaching grade 12 in India, teaching them chapter - {name}. \n
                  You are answering questions on {name}'s topic, particularly questions related to {name}'s syntax and content of the chapter. \n
                  Your responsibility is to represent a teacher, who is good in programming as well as teaching students. But, give feedback to 
                  students as faithfully as possible. Be helpful and engaging, as if talking to a future topper or a potiential breakthrough maker
                  in the tech industry. If you don't know the answer say so."""
system_prompt += f"\n\n## Summary:\n{summary}\n\n## Chapter\n{chapter}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as that favorite teacher."

In [ ]:
def chat(message, history):
    messages = [
        {"role": "system", "content": system_prompt}
    ] + history + [
        {"role": "user", "content": message}
    ]
    done = False
    while not done:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            tools=tools
        )
        finish_reason = response.choices[0].finish_reason
        if finish_reason == "tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    return response.choices[0].message.content

- We have added an `if` statement, only if the LLM (Large Language Model) wants to call a tool, which we can do.

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

### And, that's it, you have now created a chatbot that also sends notifications using Pushover. Again using the same pdf as last time talking about SQL.

### Now, its Project time!

<table>
    <tr>
        <td>
            <img src="assets/6_studyAgent.png" style="display: block;">
        </td>
        <td style="padding: 30px;">
            <h3>📚 AI Study Buddy with Urgent Reminders</h3>
            <ul>
                <li>You send your study questions or essays to the OpenAI API, and it generates summaries, answers, or explanations.</li>
                <li>If OpenAI detects something important — like you’ve missed revising a key topic before exams, or your essay draft has major gaps — it triggers the Pushover API to buzz your phone.</li>
            </ul>
            <h5><strong>Example flow:</strong></h5>
            <ol>
                <li>You paste your syllabus text into OpenAI → it analyzes and says: “These 3 topics haven’t been revised yet.”</li>
                <li>If the model marks any topic as “high priority,” your script calls Pushover:</li>
                <li>Notification: “⚠️ Don’t forget: Trigonometry & Probability still pending before tomorrow’s test.”</li>
            </ol>
        </td>
    </tr>
</table>

## Ready to move on?!

## [Ready?](../2_OpenAI/)
